In [2]:
import zipfile
import os

In [3]:
# Extract the zip file
# Create a directory for extraction
extract_dir = "/content/train"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("/content/train_SOaYf6m.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/train


In [4]:
# Extract the zip file
# Create a directory for extraction
extract_dir = "/content/test"
os.makedirs(extract_dir, exist_ok=True)

# Open and extract the zip file
with zipfile.ZipFile("/content/test.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Contents extracted to {extract_dir}")

Contents extracted to /content/test


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, random_split  # Import random_split for splitting dataset
import pandas as pd
from PIL import Image  # Import Image from PIL
import os

In [6]:
# Define transformations for the training data with data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

# Define transformations for the testing data
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [8]:
# Load the training dataset with data augmentation
train_data = datasets.ImageFolder('/content/train', transform=train_transform)

# Split the training data into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
# Custom dataset for test images listed in test.csv
class TestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.test_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.test_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.test_df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, img_name

In [11]:
# Load the test dataset from test.csv with test_transform
test_data = TestDataset(csv_file='/content/test/test/test.csv', root_dir='/content/test/test/images', transform=test_transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)



In [12]:
# Load the pre-trained ResNet18 model and modify the final layer for binary classification
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # Adjust the final layer for binary classification


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 125MB/s]


In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training the model with validation
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validate the model
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {accuracy}%")

In [ ]:
# Testing the model and preparing the results for the csv file
model.eval()
results = []
with torch.no_grad():
    for images, img_names in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(len(images)):
            image_name = img_names[i].split('/')[-1]  # Extract image name from path
            emergency_or_not = 1 if predicted[i].item() == 1 else 0
            results.append([image_name, emergency_or_not])

In [ ]:
# Create a DataFrame and save to a csv file
df = pd.DataFrame(results, columns=['image_names', 'emergency_or_not'])
df.to_csv('emergency_vehicle_results.csv', index=False)

print("Results have been saved to emergency_vehicle_results.csv")

Results have been saved to emergency_vehicle_results.csv
